## Download PBFs

In [0]:
%run "./download_pbf" $region=Malta $pbfPath=/Volumes/timo/geospatial/osm/malta-latest.osm.pbf

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


/databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: Could not open file </Workspace/Users/timo.roest@databricks.com/.bundle/pbf_datasource/dev/files/src/download_pbf> for safe execution.
  warn('Could not open file <%s> for safe execution.' % fname)


Fetching Geofabrik dataset index...


Download complete! File saved at: /Volumes/timo/geospatial/osm/malta-latest.osm.pbf


In [0]:
path_malta = "/Volumes/timo/geospatial/osm/malta-latest.osm.pbf"
filesize_mb = round(dbutils.fs.ls(path_malta)[0].size / 1048 ** 2, 1)
print(F"File size: {filesize_mb} MB")

File size: 5.8 MB


In [0]:
%run "./download_pbf" $region=France $pbfPath=/Volumes/timo/geospatial/osm/france-latest.osm.pbf

Fetching Geofabrik dataset index...


In [0]:
path_france = "/Volumes/timo/geospatial/osm/france-latest.osm.pbf"
filesize_mb = round(dbutils.fs.ls(path_france)[0].size / 1048 ** 2, 1)
print(F"File size: {filesize_mb} MB")

In [0]:
%run "./download_pbf" $region=north-america $pbfPath=/Volumes/timo/geospatial/osm/north-america-latest.osm.pbf

Fetching Geofabrik dataset index...


In [0]:
path_north_america = "/Volumes/timo/geospatial/osm/north-america-latest.osm.pbf"
filesize_mb = round(dbutils.fs.ls(path_north_america)[0].size / 1048 ** 2, 1)
print(F"File size: {filesize_mb} MB")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-2822477370419696>, line 1
----> 1 filesize_mb = round(dbutils.fs.ls("/Volumes/timo/geospatial/osm/us-latest.osm.pbf")[0].size / 1048 ** 2, 1)
      2 print(F"File size: {filesize_mb} MB")

File /databricks/python_shell/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionError: An error occurred while calling o477.ls.
: java.io.FileNotFoundException: No such file or directory /Volumes/timo/geospatial/osm/us-latest.osm.pbf
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:193)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:66)
	at com.databricks.backend.daemon.data.client

## Run Benchmark

In [0]:
%run "./load_datasource"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import time
import pandas as pd

# Create a global list to store logs
execution_logs = []

def measure_execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time

        # Determine the human-readable format of execution time
        if execution_time < 60:
            readable_time = f"{execution_time:.2f} seconds"
        elif execution_time < 3600:
            readable_time = f"{execution_time / 60:.2f} minutes"
        else:
            readable_time = f"{execution_time / 3600:.2f} hours"

        # Log the execution time
        print(f"Execution time: {readable_time}")

        # Log details in the global execution_logs list
        execution_logs.append({
            "Path": args[0] if len(args) > 0 else kwargs.get("pbfPath", "Unknown"),
            "Table Name": args[1] if len(args) > 1 else kwargs.get("tableName", "Unknown"),
            "Execution Time": readable_time
        })

        return result
    return wrapper
  
@measure_execution_time
def load_and_save_geospatial_data(pbfPath, tableName):
    (
        spark.read.format("pbf")
        .option("path", pbfPath)
        .option("geometryType", "WKT")
        .option("emptyTagFilter", True)
        .option("keyFilter", "building")
        .load()
        .write
        .mode("overwrite")
        .saveAsTable(tableName)
    )

In [0]:
load_and_save_geospatial_data("/Volumes/timo/geospatial/osm/andorra-latest.osm.pbf", 'timo.geospatial.buildings_andorra')

Execution time: 12.07 seconds


In [0]:
load_and_save_geospatial_data(path_malta, 'timo.geospatial.buildings_malta')

Execution time: 5.97 seconds


In [0]:
load_and_save_geospatial_data(path_france, 'timo.geospatial.buildings_france')

In [0]:
load_and_save_geospatial_data(path_north_america, 'timo.geospatial.buildings_north_america')

# Results

In [0]:
# Convert logs to a pandas DataFrame
execution_logs_df = pd.DataFrame(execution_logs)

# Display the DataFrame
display(execution_logs_df)

Path,Table Name,Execution Time
/Volumes/timo/geospatial/osm/andorra-latest.osm.pbf,timo.geospatial.buildings_andorra,12.07 seconds
/Volumes/timo/geospatial/osm/malta-latest.osm.pbf,timo.geospatial.buildings_malta,5.97 seconds


| Path                                                      | Table Name                              | Execution Time |
|-----------------------------------------------------------|-----------------------------------------|----------------|
| /Volumes/timo/geospatial/osm/andorra-latest.osm.pbf       | timo.geospatial.buildings_andorra       | 25.45 seconds  |
| /Volumes/timo/geospatial/osm/malta-latest.osm.pbf         | timo.geospatial.buildings_malta         | 6.95 seconds   |
| /Volumes/timo/geospatial/osm/france-latest.osm.pbf        | timo.geospatial.buildings_france        | 42.39 minutes  |
| /Volumes/timo/geospatial/osm/north-america-latest.osm.pbf | timo.geospatial.buildings_north_america | 1.25 hours     |